In [1]:
import numpy as np
from modules import bayesianL
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

def cubicarray(MM, pr=False):
    if pr: print('numero di elementi: ', len(MM))
    # In input bisogna fornire la lista dei monomi M. Ogni elemento della lista M e'
    # una lista [a,b,c] tale per cui M_i=x^{2a}y^{2b}z^{2c} e a+b+c=n, dove n e' il grado del monomio.
    # I monomi in input comprendono tutte le permutazioni.
    # Voglio ora applicare le permutazioni x,y,z in modo da simmetrizzare i monomi e ottenere le armoniche cubiche.
    lenM = len(MM)
    M=[]
    #Converto M a una lista di liste
    for i in range(len(MM)):
        M.append(list(MM[i]))
    Mt = copy.copy(M)
    # devo prima capire nella base dei monomi quali sono legati da permutazione. posso nominarli.
    nameC = []
    conta = 0
    contaci = 0
    if pr: print('numero cicli teornicamente necessari', int(len(M) ** 2 * 3 / 2))
    for i in range(lenM):
        rep = M[i]
        # rep (M[i]) e' il mio rappresentante nella lista,
        # voglio vedere quali altri elementi sono equivalenti per permutazione a rep
        nameC_i = []
        if not (rep in Mt): continue
        # se rep non sta nella lista temporanea Mt significa che e' equivalente a un elemento precedente: salto il ciclo
        nameC_i.append(np.array(rep))
        # in nameC_i inserisco gli elementi equivalenti per simmetria a rep
        for j in range(i + 1, lenM):
            # parto a cercare dall'elemento successivo nella lista, gli altri li ho gia' sicuramente controllati.
            Mjt = list(copy.copy(M[j]))
        
            if rep[0] in Mjt:
                Mjt.remove(rep[0])
                if rep[1] in Mjt:
                    Mjt.remove(rep[1])
                    if rep[2] in Mjt:
                        # rep e M[j] sono simmetrici per permutazione!
                        nameC_i.append(np.array(M[j]))
                        Mt.remove(M[j])
                        
                        # rimuovo da Mt l'elemento M[j] che e' simmetrico per permutazione a rep
                        conta += 1
        contaci += len(nameC_i)
        nameC.append(nameC_i)
        # aggiungo la lista degli elementi associati a M[i], nameC_i, alla lista degli elementi NON equivalenti, nameC!
    if contaci != len(M): raise ValueError
    if pr: print('numero cicli compiuti', conta)
    if pr: print('numero di elementi non equivalenti per simmetria cubica: ', len(nameC))
    # in output c'e' la lista degli elementi che formano una classe di equivalenza.
    return nameC

def datadicG(data, sigmadata, G):

    dicdata = {}
    
    dicsigma = {}
    
    for i in range(len(G)):
        dicdata[tuple(G[i])] = data[i]
        dicsigma[tuple(G[i])] = sigmadata[i]
        
    return dicdata, dicsigma

def datiplot(C, dic, dics):
    gplot=[]
    dataplot=[]
    datasigmaplot=[]
    for i in range(len(C)):
        gplot.append(np.linalg.norm(C[i][0]))
        dataav=0
        datasigma=0
        for j in range(len(C[i])):
            dataav+=dic[tuple(C[i][j])]/len(C[i])  
            datasigma+=(dics[tuple(C[i][j])]/len(C[i])  )**2
        dataplot.append(dataav)
        datasigmaplot.append(np.sqrt(datasigma))
    return np.array(gplot), np.array(dataplot), np.array(datasigmaplot)



In [2]:
data=np.transpose(np.loadtxt('/Users/enricodrigo/Documents/LAMMPS/tpc30ns3375/nlttk.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[0. 0. 1.] [1. 0. 0.] [0. 1. 0.]


In [3]:
mN, SN, y_infer, sy_infer=bayesianL.bayesianpol(grid,sdata,12,200,0.001,grid,True)

parametri ottimali [  0.5554527  -18.21726707  15.21825071  74.64502826   8.5390216
  56.00742766  17.53189879 -14.66491996  85.06745701 -34.37627479
  -3.2738066  156.10455046  13.42204688  26.66743393  88.17314559
   4.52328597 -26.01742919 -13.72720063 -16.14700532  11.58826412
 -55.38757744  19.41485723  27.84690006  -8.21900877  10.28867548
 -50.71972949 -88.35945138 -30.2997949   21.66233925  60.4489216
  13.86817726]
numero di armoniche cubiche 19
valore a kmin inferito 0.5137340503310085 bias 0.5554526955835611 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 0.006478899954671001


In [4]:
N=100
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=bayesianL.bestfit(grid, sdata, N, grid, False, True)

parametri ottimali [   0.8368631   -34.49955449   25.698178    538.58331262 -105.34921635
  125.27979778 -164.57441702  112.16088156   16.7047552   272.33519646
 -163.96819618 -189.6619525  -262.12672468  226.87013708 -165.47661183
  225.89138642  -98.02588754   99.02825421 -101.33199162 -103.61132244
  294.8317675   -57.42855976   79.28061217 -177.98627394   56.86518567]
numero di armoniche cubiche 14
valore a kmin inferito 0.6726150930788569 bias 0.8368631023892266 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 3.998813775295752e-05
grado ottimale 20 grado massimo tentato 28


In [5]:
%matplotlib widget
plt.plot(log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
C=cubicarray(list(grid), True)
dic, dics=datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=datiplot(C, dic, dics)
C=cubicarray(grid[:N], True)
dic, dics=datadicG(y_infer, np.sqrt(sy_infer), grid[:N])
gplotfit, dataplotfit, datasigmaplotfit=datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(gplotfit, dataplotfit+datasigmaplotfit, dataplotfit-datasigmaplotfit,  alpha=0.3, color='red')
plt.plot(gplotfit, dataplotfit, '.-', color='red')
plt.errorbar(gplot, dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118
numero di elementi:  100
numero cicli teornicamente necessari 15000
numero cicli compiuti 72
numero di elementi non equivalenti per simmetria cubica:  28


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [7]:
pred=[]
spred=[]
Npoints=[]
r=0
for N in range(60, 140):
    
    if r==np.linalg.norm(grid[N]): continue
    r=np.linalg.norm(grid[N+1])
    mN, SN, y_infer, sy_infer, spar, log_evidence_vP=bayesianL.bestfit(grid, sdata, N+1, grid, False, True)
    Npoints.append(np.linalg.norm(grid[N+1]))
    print(r, grid[N+1], N+1)
    pred.append(mN[0])
    spred.append(SN[0,0])

parametri ottimali [  0.77774206 -12.70621291   8.81686641   3.68644101   2.06169697
   0.39058681   6.24996406  -4.81184883  -0.84418387  -4.3141487
  -1.5952245   -0.10416086  -3.25248243   1.73842055   4.62723516
   1.75159095]
numero di armoniche cubiche 8
valore a kmin inferito 0.6533096846667465 bias 0.7777420573248567 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 388.24889336821275
grado ottimale 14 grado massimo tentato 28
4.123105625617661 [1. 4. 0.] 61


parametri ottimali [ 9.08798871e-01 -2.62320489e+01  8.78174835e+01 -1.13415148e+03
 -4.80911659e+01 -2.93553946e+02 -3.47946237e+01  3.92889044e+01
 -9.82089125e+01  1.23191873e+01 -5.97259329e+00 -1.01022966e+01
 -1.47663964e-02]
numero di armoniche cubiche 7
valore a kmin inferito 0.6902957247979573 bias 0.9087988709361281 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 2.7633319219381736e+16
grado ottimale 12 grado massimo tentato 28
4.242640687119285 [3. 3. 0.] 63


parametri ottimali [  0.76502504 -13.64874781   8.556546     5.74472098   4.96268448
   0.91152911   9.14365079  -5.48491982  -0.68483766  -6.46383913
  -8.90241062   2.02568356  -7.84779349  13.15363175  15.57792172
   8.55297922  -4.74250992  -9.57906867  -2.26788078]
numero di armoniche cubiche 10
valore a kmin inferito 0.6488017565693986 bias 0.7650250427240255 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 6657047.564805969
grado ottimale 16 grado massimo tentato 28
4.358898943540674 [1. 3. 3.] 69


parametri ottimali [  0.77549754 -15.12148793   9.11902026  -1.0079349   11.62898746
   2.76235302  15.40098411 -13.70987655   9.42230826 -17.99258139
 -15.4747426   14.05919758  -8.59091757  24.39450319   2.58980212
  21.39763334  -7.23310538   8.70032155  -7.57717263]
numero di armoniche cubiche 10
valore a kmin inferito 0.653632376299565 bias 0.7754975413150287 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 800275.3273902357
grado ottimale 16 grado massimo tentato 28
4.47213595499958 [0. 2. 4.] 72


parametri ottimali [  0.78429282 -17.52394318  11.22419075  -4.63248663  15.09631044
   1.23607008  26.54102393 -23.45525056  -6.86042688 -36.65144675
  -5.25970813 -11.20475508  -6.05275377  20.1768002  -20.00786242
  18.57277404   8.97442327  -9.26154484   8.55403839 -10.93657002
   9.81250772 -10.41942886]
numero di armoniche cubiche 12
valore a kmin inferito 0.6564745148862464 bias 0.7842928234959039 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 6.519775643415926
grado ottimale 18 grado massimo tentato 28
4.58257569495584 [4. 1. 2.] 78


...

parametri ottimali [   0.7297934   -22.23061995   13.03590703 -329.23876597    3.50175328
  -18.0890063     6.53670398  -22.51886867  -11.32775903  -11.12428763
   -2.76115414  184.27750914  -25.67679305   30.10537188  345.63495663
   40.99901921   21.1912678   -15.8587664    31.54864087  -21.33185535
  -61.61587352  -28.52845765]
numero di armoniche cubiche 12
valore a kmin inferito 0.6276319086352652 bias 0.7297933997593941 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 17178.535992239857
grado ottimale 18 grado massimo tentato 28
5.196152422706632 [1. 1. 5.] 111


parametri ottimali [  0.67162351 -17.13013597  10.66781118 -24.2956638   17.50424996
  41.9895286   34.75997425 -33.79113526  73.54887098 -66.43908668
   8.76974271  58.47281805  24.80838519  23.52718149  48.50948794
  25.507995     2.97205163  44.43026514   5.73397376 -20.44998199
  -2.8637906  -31.85387898   8.91306718  -9.29165022  13.4752961 ]
numero di armoniche cubiche 14
valore a kmin inferito 0.5941353997284272 bias 0.6716235093392084 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 982.9242177673337
grado ottimale 20 grado massimo tentato 28
5.385164807134504 [3. 4. 2.] 115


parametri ottimali [   0.69246128  -21.87690919    7.81978189  228.18893076   26.1942159
  -32.38724782   37.34206687  -67.76123971   13.01510214 -103.4232656
   43.30629482 -112.78139971   68.51707855   39.87810284 -174.09726024
   37.05934492   -4.27320678   21.54706735  -21.6311372   -45.0526406
   98.53276445  -47.51342133   25.18557581  -56.90137826   30.75600372]
numero di armoniche cubiche 14
valore a kmin inferito 0.6058634163046077 bias 0.692461283809694 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 534.0275765977916
grado ottimale 20 grado massimo tentato 28
5.477225575051661 [5. 1. 2.] 127


parametri ottimali [ 7.17934487e-01 -2.90705907e+01  1.01845280e+02 -3.07994446e+03
 -5.36657802e+01 -2.86509692e+02 -5.49181806e+01  4.90927822e+01
 -3.40658733e+02  4.23184151e+01 -1.87657979e+01  1.41380057e+02
 -1.81385817e+01  3.17251046e+00 -2.26539027e+01  3.04241755e+00]
numero di armoniche cubiche 8
valore a kmin inferito 0.6151448258676274 bias 0.7179344870770352 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 3.408515139252462e+18
grado ottimale 14 grado massimo tentato 28
5.656854249492381 [0. 4. 4.] 133


parametri ottimali [ 7.14627389e-01 -2.93727781e+01  9.60904310e+01 -2.93276504e+03
 -4.52106682e+01 -8.22664084e+02 -7.05779246e+01  5.99816964e+01
 -6.89496521e+02  4.68121918e+01 -3.19013040e+01  5.89902708e+02
 -1.79323576e+01  5.06658536e+00 -9.00920573e+01  3.97747819e+00]
numero di armoniche cubiche 8
valore a kmin inferito 0.613762826552636 bias 0.7146273889300574 dato a kmin 0.7160646415940555
determinante matrice delle armoniche cubiche ridotte: 771976160481300.4
grado ottimale 14 grado massimo tentato 28
5.744562646538029 [2. 5. 2.] 136


In [8]:
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')

ax[0].errorbar(np.array(Npoints), np.array(pred), np.sqrt(np.array(spred)), fmt='.')
    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
#ax[1].set_title('Best fit at k=0 with data in range $[k_{min}: k_{max}]$')
#
#for i in range(1,len(Ndat)):
#    #ax.plot(sdata[0][Ndat[i]]*10, mNs[Ndat[i]][0], 'o-')
#    ax[1].errorbar(sdata[0][Ndat[i]]*10,resN[i][1], resN[i][2], fmt='o')
#    
#ax[1].set_xlabel('$k_{max} (nm^{-1})$')
#ax[1].set_ylabel('prediction at k=0')
#ax[1].legend()
ax[1].set_title('Data')

ax[1].errorbar(gplot, dataplot,datasigmaplot, fmt='.', label='3375 water molecules at 300k 30ns') 

ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')
ax[1].set_ylabel(r'$\frac{E}{\triangledown(T)}$ (V/K)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\frac{E}{\\triangledown(T)}$ (V/K)')

In [39]:
mr, sr, logevidence= bayesianL.bayesianmodelprediction(grid, sdata, N, grid, False, True)

model average prediction 0.0028915591787525984 4.835424546097271e-05


In [46]:
for i in range(len(grid)):
    print(grid[i],np.linalg.norm(grid[i]), i)

[0. 0. 1.] 1.0 0
[1. 0. 0.] 1.0 1
[0. 1. 0.] 1.0 2
[0. 1. 1.] 1.4142135623730951 3
[1. 0. 1.] 1.4142135623730951 4
[1. 1. 0.] 1.4142135623730951 5
[1. 1. 1.] 1.7320508075688772 6
[0. 0. 2.] 2.0 7
[0. 2. 0.] 2.0 8
[2. 0. 0.] 2.0 9
[2. 0. 1.] 2.23606797749979 10
[0. 1. 2.] 2.23606797749979 11
[1. 2. 0.] 2.23606797749979 12
[1. 0. 2.] 2.23606797749979 13
[2. 1. 0.] 2.23606797749979 14
[0. 2. 1.] 2.23606797749979 15
[1. 1. 2.] 2.449489742783178 16
[2. 1. 1.] 2.449489742783178 17
[1. 2. 1.] 2.449489742783178 18
[2. 2. 0.] 2.8284271247461903 19
[2. 0. 2.] 2.8284271247461903 20
[0. 2. 2.] 2.8284271247461903 21
[1. 2. 2.] 3.0 22
[2. 2. 1.] 3.0 23
[0. 3. 0.] 3.0 24
[0. 0. 3.] 3.0 25
[3. 0. 0.] 3.0 26
[2. 1. 2.] 3.0 27
[1. 3. 0.] 3.1622776601683795 28
[3. 1. 0.] 3.1622776601683795 29
[1. 0. 3.] 3.1622776601683795 30
[0. 3. 1.] 3.1622776601683795 31
[0. 1. 3.] 3.1622776601683795 32
[3. 0. 1.] 3.1622776601683795 33
[1. 1. 3.] 3.3166247903554 34
[1. 3. 1.] 3.3166247903554 35
[3. 1. 1.] 3.3166247903

In [51]:
C

[[array([0., 0., 1.]), array([1., 0., 0.]), array([0., 1., 0.])],
 [array([0., 1., 1.]), array([1., 0., 1.]), array([1., 1., 0.])],
 [array([1., 1., 1.])],
 [array([0., 0., 2.]), array([0., 2., 0.]), array([2., 0., 0.])],
 [array([2., 0., 1.]),
  array([0., 1., 2.]),
  array([1., 2., 0.]),
  array([1., 0., 2.]),
  array([2., 1., 0.]),
  array([0., 2., 1.])],
 [array([1., 1., 2.]), array([2., 1., 1.]), array([1., 2., 1.])],
 [array([2., 2., 0.]), array([2., 0., 2.]), array([0., 2., 2.])],
 [array([1., 2., 2.]), array([2., 2., 1.]), array([2., 1., 2.])],
 [array([0., 3., 0.]), array([0., 0., 3.]), array([3., 0., 0.])],
 [array([1., 3., 0.]),
  array([3., 1., 0.]),
  array([1., 0., 3.]),
  array([0., 3., 1.]),
  array([0., 1., 3.]),
  array([3., 0., 1.])],
 [array([1., 1., 3.]), array([1., 3., 1.]), array([3., 1., 1.])],
 [array([2., 2., 2.])],
 [array([0., 3., 2.]),
  array([3., 0., 2.]),
  array([3., 2., 0.]),
  array([2., 3., 0.]),
  array([2., 0., 3.]),
  array([0., 2., 3.])],
 [array